In [2]:
!pip install -q datasets==2.14.6 transformers
!pip install -q fsspec==2023.10.0 gcsfs==2023.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_m

In [3]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1")
train_data = dataset["train"]
val_data = dataset["validation"]

print(f"✅ Loaded: {len(train_data)} training samples")
print(f"🧾 Example: {train_data[0]}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

✅ Loaded: 36718 training samples
🧾 Example: {'text': ''}


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Disable WandB and other logging that asks for API keys
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_WANDB"] = "1"

model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    eval_strategy="epoch", # Changed from evaluation_strategy to eval_strategy
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_strategy="no",  # don’t save to hub
    report_to=[],        # no WandB, no TensorBoard
)

# Set the padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(200)),  # small subset to test quickly
    eval_dataset=tokenized_val.select(range(100)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-5-484968204.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.912600,3.708437


TrainOutput(global_step=50, training_loss=4.044103546142578, metrics={'train_runtime': 13.023, 'train_samples_per_second': 15.357, 'train_steps_per_second': 3.839, 'total_flos': 15087573504000.0, 'train_loss': 4.044103546142578, 'epoch': 1.0})

In [9]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

import torch
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def compute_top_k_accuracy(dataset, model, tokenizer, k=5, num_samples=100):
    model.eval()
    correct = 0
    total = 0

    for sample in tqdm(dataset.select(range(num_samples))):
        text = sample["text"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        input_ids = inputs["input_ids"].to(model.device)

        # Skip short inputs
        if input_ids.shape[1] < 2:
            continue

        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits

        # Get the predicted logits for each token in the sequence
        for i in range(input_ids.shape[1] - 1):
            next_token_logits = logits[0, i]
            true_next_token = input_ids[0, i + 1]

            # Top-k predictions
            top_k_tokens = torch.topk(next_token_logits, k).indices

            if true_next_token in top_k_tokens:
                correct += 1
            total += 1

    accuracy = correct / total if total > 0 else 0
    return accuracy

top_k_acc = compute_top_k_accuracy(val_data, model, tokenizer, k=5, num_samples=100)
print(f"Top-5 Accuracy: {top_k_acc * 100:.2f}%")


Perplexity: 40.79


100%|██████████| 100/100 [00:01<00:00, 59.06it/s]

Top-5 Accuracy: 58.76%


In [20]:
input_text = "he opened the door and saw"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

with torch.no_grad():
    output = model.generate(input_ids, max_new_tokens=1, do_sample=True)

predicted_text = tokenizer.decode(output[0])
print("Next word prediction:", predicted_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Next word prediction: he opened the door and saw me
